In [75]:
import numpy as np
from scipy.optimize import minimize,differential_evolution
import random
import time
import itertools
import matplotlib.pyplot as plt
import copy

In [3]:
%load_ext cython

In [4]:
%%cython
import numpy as np
cdef float correlations(list sequence,int k):
    return sum([sequence[i]*sequence[i+k] for i in range(0,len(sequence)-k)])

cdef float energy(list sequence):
    return sum([correlations(sequence,k)**2 for k in range(1,len(sequence))])

cpdef float merit_factor(list sequence):
    return len(sequence)**2/(2*energy(sequence))

cpdef float mae(list sequence):
    cpdef int k=1
    cpdef list error_list=[]
    for k in range(1,len(sequence),2):
        stat=[abs(sequence[i+k]-sequence[i]) for i in range(len(sequence)-k)]
        error_list.append(abs(sum(stat)-(len(sequence)-k)))
    return np.sum(np.array(error_list))

cpdef both(list sequence):
    return (0.5*mae(sequence)+0.5*merit_factor(sequence))

In [72]:
def generate_sequence(N):        
    return [1 if np.random.rand()>0.5 else -1 for i in range(N)]

def get_nb_swap(sequence):
    return np.sum([1 for i in range(len(sequence)-1) if sequence[i]!=sequence[i+1]])

def stat_diff(sequence):
    list_diff=[0]
    for i in range(1,len(sequence)):
        nb_dif=np.sum(np.array([abs(sequence[j+i]-sequence[j]) for j in range(0,len(sequence)-i)]))
        list_diff.append(nb_dif-np.ceil((len(sequence)-i)/2))
    return np.mean(list_diff)

def sbm(x,nb_flip=1):
    index_flip=set([int(np.random.rand()*len(x)) for i in range(nb_flip)])
    y=x[:]
    y=[y[i] if i not in index_flip else -y[i] for i in range(0,len(x))]
    return y

def RLS(N,epochs,nb_flip=1,optimizer=merit_factor):
    history=[]
    x=generate_sequence(N)
    fx=optimizer(x)
    for i in range(epochs):
        y=sbm(x,nb_flip)
        fy=optimizer(y)
        if fy>=fx:
            x=y[:]
            fx=fy
        history.append(merit_factor(x))
    return(history)    

def RLS_restart(N,epochs,restart=True,no_change_threshold=100):
    history=[]
    x=generate_sequence(N)
    fx=merit_factor(x)
    no_change=0
    for i in range(epochs):
        y=sbm(x)
        fy=merit_factor(y)
        if fy>=fx:
            x=y[:]
            fx=fy
            no_change=0
        elif fy<fx:
            no_change+=1
            if no_change==no_change_threshold:
                if restart==True:
                    x=generate_sequence(N)
                else:
                    index_flip=set([int(np.random.rand()*N) for i in range(restart)])
                    x=[-x[i] if i in index_flip else x[i] for i in range(0,N)]

                fx=merit_factor(x)
                no_change=0
                
        history.append(fx)
    return(history)    

def EA(N,p,epochs):
    history=[]
    x=generate_sequence(N)
    fx=merit_factor(x)
    for i in range(epochs):
        y=[-x[i] if np.random.rand()<p else x[i] for i in range(N)]
        fy=merit_factor(y)
        if fy>=fx:
            x=y[:]
            fx=fy
        history.append(fx)
    return(history)

def simulated_annealing(N,nb_of_flip,nb_iterations,t_max,t_growth="linear"):
    history=[]
    x=generate_sequence(N)
    t=1
    t_max=t_max
    best_solution=x
    best_solution_merit=-np.inf
    fx=merit_factor(x)
    step=0
    for i in range(nb_iterations):
        index_flip=set([int(np.random.rand()*N) for i in range(nb_of_flip)])
        y=x[:]
        y=[y[i] if i not in index_flip else -y[i] for i in range(0,N)]
        fy=merit_factor(y)
        if fy>=fx:
            x=y[:]
            fx=fy
            if fx>best_solution_merit:
                best_solution=y[:]
                best_solution_merit=fy
        elif np.random.rand()<=np.exp((fy-fx)*t):
            x=y[:]
            fx=fy
        history.append(fx)
        if t_growth=="linear":
            t+=t_max/nb_iterations
        elif t_growth=="exp":
            step+=np.log(t_max)/nb_iterations
            t=np.exp(step)
        elif t_growth=="log":
            if step<1:
                step=1
            step+=np.exp(t_max)/nb_iterations
            t=np.log(step)
#     print(t)
    return history

def genetic(N,nb_iterations,population_size,mutation_ratio,crossover_ratio):
    history=[]
    population=[generate_sequence(N) for i in range(population_size)]
    crossover_length=int(N*crossover_ratio)
    for step in range(40):
        random.shuffle(population)
        population=[sbm(population[i]) for i in range(0,int(len(population)*mutation_ratio))]
        couples=[[i,j] for j in range(0,crossover_length) for i in range(j,crossover_length) if i!=j]
        tmp=[crossover(population[i[0]],population[i[1]]) for i in couples]
        population+=[i[0] for i in tmp]+[i[1] for i in tmp]
        merit_factor_population=[[i,merit_factor(population[i])] for i in range(len(population))]
        merit_factor_population=sorted(merit_factor_population,key=lambda x :x[1],reverse=True)
        merit_factor_population=merit_factor_population[:100]
        population=[population[merit_factor_population[i][0]] for i in range(len(merit_factor_population))]
        history.append(merit_factor_population[0][1])
    return history

class Tree(object):
    def __init__(self,N,id_tree=None,init_sequence=None,parent=None,parent_list=None,merit_factor_back_method=None):
        self.id=id_tree
        if init_sequence==None:
            self.sequence = generate_sequence(N)
        if init_sequence!=None:
            self.sequence = init_sequence
        
        self.merit_factor_true=stat_diff(self.sequence)
        self.merit_factor = merit_factor(self.sequence)#-stat_diff(self.sequence)/5
        self.merit_factor_back = self.merit_factor
        self.children=[]
        self.children_merit={self.id:self.merit_factor}
        self.parent=None
        self.merit_factor_back_method=merit_factor_back_method
        self.nb_visit=1
        
        if parent!=None:
            self.parent=parent
            self.update_parent_merit()
            self.uct = 0 
        else:
            self.uct=0
    
    def update_parent_merit(self):
        x=self
        while x.parent!=None:
            x.parent.children_merit[x.id] = x.merit_factor_back
            if self.merit_factor_back_method=="min":
                x.parent.merit_factor_back = np.min(list(x.parent.children_merit.values()))
            elif self.merit_factor_back_method=="max":
                x.parent.merit_factor_back = np.max(list(x.parent.children_merit.values()))
            elif self.merit_factor_back_method=="mean":
                x.parent.merit_factor_back = np.mean(list(x.parent.children_merit.values()))
            elif self.merit_factor_back_method=="median":
                x.parent.merit_factor_back = np.median(list(x.parent.children_merit.values()))
            x=x.parent
            x.nb_visit+=1
    
    def get_uct(self):
        if self.parent==None:
            return 0
        if self.children==[]:
            return self.merit_factor
        self.uct=self.merit_factor_back/(self.nb_visit)+np.sqrt(2*np.log(self.parent.nb_visit)/(self.nb_visit))
        return self.uct 

def monte_carlo(N,depth,width,merit_factor_back_method="mean",choose_children="UCT"):
    x=Tree(N,id_tree=0)
    tree_list=[x]
    for i in range(depth):
        max_leaf=tree_list[0]
        if choose_children=="UCT":
            max_leaf=max(tree_list,key=lambda x:x.get_uct())
        elif choose_children=="merit_factor":
            max_leaf=max(tree_list,key=lambda x:x.merit_factor)
        elif choose_children=="merit_factor_back":
            max_leaf=max(tree_list,key=lambda x:x.merit_factor_back)
        for j in range(width):
            new_leaf=Tree(N,id_tree=len(tree_list),init_sequence=sbm(max_leaf.sequence),
                          parent=max_leaf,merit_factor_back_method=merit_factor_back_method)
            max_leaf.children.append(new_leaf)
            tree_list.append(new_leaf)
    return [i.merit_factor for i in tree_list]

def print_tree(tree_list):
    for i in tree_list:
        for j in i.children:
            print("\"(id=%d,merit_factor=%.2f,merit_factor_back=%.2f)\" -> \"(id=%d,merit_factor=%.2f,merit_factor_back=%.2f)\" "%
                  (i.id,round(i.merit_factor,2),round(i.merit_factor_back,2),j.id,round(j.merit_factor,2),round(j.merit_factor_back,2)))


def analyse(epochs,nb_run,N_interval):
    np.random.seed(1)
    dict_rls={"name":"RLS"}
    dict_rls_restart={"name":"RLS_restart"}
    dict_ea={"name":"EA"}
    dict_sa={"name":"Simulated Annealing"}
    dict_monte_carlo={"name":"Monte_Carlo"}
    for size in N_interval:
        for run in range(nb_run):
            result_rls=RLS(size,epochs)
            result_rls_restart=RLS_restart(size,epochs,restart=2)
            result_ea=EA(size,1/size,epochs)
            result_sa=simulated_annealing(size,1,epochs,t_max=15,t_growth="log")
            result_monte_carlo=monte_carlo(size,depth=int(epochs/10),width=10)
            if run==0:
                dict_rls[size]=[result_rls]
                dict_rls_restart[size]=[result_rls_restart]
                dict_ea[size]=[result_ea]
                dict_sa[size]=[result_sa]
                dict_monte_carlo[size]=[result_monte_carlo]
            else:
                dict_rls[size].append(result_rls)
                dict_rls_restart[size].append(result_rls_restart)
                dict_ea[size].append(result_ea)
                dict_sa[size].append(result_sa)
                dict_monte_carlo[size].append(result_monte_carlo)
    return dict_rls,dict_rls_restart,dict_ea,dict_sa,dict_monte_carlo

In [74]:
dict_rls,dict_rls_restart,dict_ea,dict_sa,dict_monte_carlo=analyse(2000,20,N_interval=[i for i in range(10,200,15)])